In [1]:
# !pip install selenium

In [2]:
# !pip install undetected_chromedriver

In [2]:
# вот эта штука не оставляет следов по которым всякие антиботы
# могут нас поймать
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from urllib.parse import unquote
from bs4 import BeautifulSoup
import random
from tqdm.notebook import tqdm
import re
import time
import json
import pandas as pd

In [3]:
def open_linkedin():
    #в этом файле мои логин и пароль
    with open('linkedin_login.txt', 'r') as handler:
        lines = handler.readlines()
        MAIL, PASS = lines[0].strip(), lines[1]

    caps = DesiredCapabilities().CHROME
    caps['pageLoadStrategy'] = 'eager'
    driver = webdriver.Chrome()
    time.sleep(50)
    driver.get("https://linkedin.com/uas/login")
    print('driver.get логин - выполнено')
    login_field = driver.find_element(By.ID,"username")
    login_field.send_keys(MAIL)
    pass_field = driver.find_element(By.ID,"password")
    pass_field.send_keys(PASS)
    pass_field.submit()
    time.sleep(4)
    print('вход выполнен')
    
    return driver

In [4]:
driver = open_linkedin()

driver = webdriver.Chrome() определено
driver.get логин - выполнено
вход выполнен


In [24]:
driver.current_url

'https://www.linkedin.com/search/results/content/?keywords=yandex&origin=SWITCH_SEARCH_VERTICAL&sid=cQb'

In [571]:
#прокручивает всплывающее окно пока высота не прекратит меняться
def scroll_reactions_window():
    js_code = "return document.getElementsByClassName('artdeco-modal__content')[0].scrollHeight"
    last_height = driver.execute_script(js_code)
    while True:
        time.sleep(random.uniform(1,3))
        scroll_script = 'arguments[0].scrollTop = arguments[0].scrollHeight'
        driver.execute_script(scroll_script, driver.find_element(By.CLASS_NAME,'artdeco-modal__content'))
        time.sleep(random.uniform(2,4))
        new_height = driver.execute_script(js_code)
        if new_height == last_height:
            print('прокрутка остановлена')
            break
        last_height = new_height

прокрутка остановлена


проверяем работу скрипта

In [707]:
#пустой словарь куда будем скидывать результаты
result_dict = {}

# переходим на страницу с нужным поисковым запросом
driver.get(f'https://www.linkedin.com/search/results/content/?keywords=преподавание%20it&origin=SWITCH_SEARCH_VERTICAL')
time.sleep(2)

#создаем 2 списка, в каждом из них элементы при нажатии на которые открываются лайки
likes_list_1 = driver.find_elements(By.CLASS_NAME, 'social-details-social-counts__reactions-count')
likes_list_2 = driver.find_elements(By.CLASS_NAME, 'social-details-social-counts__social-proof-text')

#каунтер обозначает номер поста, 2 следующих указателя определяют номера элементов реакций
count = 0
reactions_button_counter_1 = 0
reactions_button_counter_2 = 0
# переменная soup отвечает за содержимое всей страницы
soup = BeautifulSoup(driver.page_source,'lxml')

In [708]:
while True:
    time.sleep(random.uniform(2,4))
    # переменная post_card отвечает за содержимое карточки поста
    post_card = soup.find_all('div','pt1 mb2 artdeco-card')[count]
    try:
        #пытаемся вытащить текст поста
        text_relative_feed = post_card.find('div','update-components-text relative feed-shared-update-v2__commentary')
        post_text = text_relative_feed.find('span',{'dir':'ltr'}).text
        result_dict[count] = [post_text,{}]
        print(post_text[:20])
    except:
        #если текста нет и в посте только картинка, то так и пишем
        result_dict[count] = ['Just image in post',{}]
        print('Just image in post')
    
    #проверяем условие по которому делается клик на соответствующую кнопку с лайками
    if 'social-details-social-counts__reactions-count' in str(post_card):
        likes_list_1[reactions_button_counter_1].click()
        reactions_button_counter_1 += 1
    elif 'social-details-social-counts__social-proof-text' in str(post_card):
        likes_list_2[reactions_button_counter_2].click()
        reactions_button_counter_2 += 1
    else:
        count += 1
        print('error')
        continue
    
    scroll_reactions_window()
    
    time.sleep(3)
    #кнопка с лайками нажата, парсим всю инфу со страницы в новый суп
    reactions_window_soup = BeautifulSoup(driver.page_source,'lxml')
    #все лайкнувшие юзеры будут тут, но нужно скролить до конца
    reactions_window_soup = reactions_window_soup.find('ul','artdeco-list artdeco-list--offset-1')
    usernames = []
    user_links = []
    job_titles = []
    for i in reactions_window_soup.find_all('a', 'link-without-hover-state ember-view'):
        try:
            name = i.find('span',{'dir':'ltr'}).text
        except:
            name = 'Not defined'
        usernames.append(name)
        user_link = i['href']
        user_links.append(user_link)
        job_title = i.find('div','artdeco-entity-lockup__caption ember-view').text.strip()
        job_titles.append(job_title)
        
    # упаковываем результаты в словарь
    for i,v in enumerate(user_links):
        result_dict[count][1][v] = [usernames[i],job_titles[i]]
    #закрываем страницу с лайками и добавляем к каунтеру 1
    driver.find_element(By.CLASS_NAME, 'artdeco-button__icon').click()
    count += 1
    #скролим если спарсили 9 постов
    if counter !=0 and counter % 9 == 0:
#         print('scroll')
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(random.uniform(2,5))
        #создаем 2 списка, в каждом из них элементы при нажатии на которые открываются лайки
        likes_list_1 = driver.find_elements(By.CLASS_NAME, 'social-details-social-counts__reactions-count')
        likes_list_2 = driver.find_elements(By.CLASS_NAME, 'social-details-social-counts__social-proof-container')
        soup = BeautifulSoup(driver.page_source,'lxml')

Преподавание в Высше
прокрутка остановлена
С удовольствием сооб
прокрутка остановлена
Из ТГ "Будни рекруте
error
7 грехов современных
error
"В состав факультета
error
Всем привет! 👋Кажетс
error
Когда мои ученики из
error
Как продвигать бизне
error
Уважаемые QA джуны и
error
Приветствую участник
прокрутка остановлена
Я учился в Высшей Шк
прокрутка остановлена
Теперь я спикер Ozon
прокрутка остановлена
Кто меня удивил боль
прокрутка остановлена
Ekaterina Kuznetsova
прокрутка остановлена
Знаешь английский, н
прокрутка остановлена
🌀 Учебному центру DT
прокрутка остановлена
Замечательные новост
прокрутка остановлена
Продолжая идею полез
прокрутка остановлена
Мой курс PR в IT уже
прокрутка остановлена
Хей, актуальный вопр
прокрутка остановлена
Навел меня сегодня о
прокрутка остановлена
Видео-пост благодарн
прокрутка остановлена
Какие есть способы п
прокрутка остановлена
Почему с нейтивом не
прокрутка остановлена
 ✔ Преимущества рабо
прокрутка остановлена
Если вас зовут Серге
прокрутка остано

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".artdeco-modal__content"}
  (Session info: chrome=114.0.5735.134); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0109A813+48355]
	(No symbol) [0x0102C4B1]
	(No symbol) [0x00F35358]
	(No symbol) [0x00F609A5]
	(No symbol) [0x00F60B3B]
	(No symbol) [0x00F8E232]
	(No symbol) [0x00F7A784]
	(No symbol) [0x00F8C922]
	(No symbol) [0x00F7A536]
	(No symbol) [0x00F582DC]
	(No symbol) [0x00F593DD]
	GetHandleVerifier [0x012FAABD+2539405]
	GetHandleVerifier [0x0133A78F+2800735]
	GetHandleVerifier [0x0133456C+2775612]
	GetHandleVerifier [0x011251E0+616112]
	(No symbol) [0x01035F8C]
	(No symbol) [0x01032328]
	(No symbol) [0x0103240B]
	(No symbol) [0x01024FF7]
	BaseThreadInitThunk [0x773400C9+25]
	RtlGetAppContainerNamedObjectPath [0x77A57B4E+286]
	R

In [709]:
len(result_dict)

116

In [488]:
'''структура словаря

result_dict = {
    'count_number_1':['post_1_text',{
        'user_link_1':['username_1_1','user_job_title'],
        'user_link_2':['username_1_2','user_job_title']
    }],
    'count_number_2':['post_2_text',{
        'user_link_1':['username_2_1','user_job_title'],
        'user_link_2':['username_2_2','user_job_title']
    }]
}
'''

SyntaxError: invalid syntax (4019590223.py, line 1)

In [710]:
with open("datasets/prepodavanie_it_final.json", "w") as outfile:
    json.dump(result_dict, outfile)

In [711]:
df = pd.read_json('datasets/prepodavanie_it_final.json', orient='index')
df.columns = ['post_text','users']
df

,post_text,users
0,Преподавание в Высшей школе экономики на курсе...,{'https://www.linkedin.com/in/ACoAABiJ2BgBn3Qe...
1,"С удовольствием сообщаю, что я начинаю препода...",{'https://www.linkedin.com/in/ACoAADoCp0kBt75T...
2,"Из ТГ ""Будни рекрутера"" @recruiter_liveНавеяно...",{}
3,7 грехов современных бизнесменов и менеджеров....,{}
4,"""В состав факультета войдут несколько кафедр. ...",{}
...,...,...
111,Senior Lecturer/Teaching Professor (Критическо...,{'https://www.linkedin.com/in/ACoAABr9uBYB428_...
112,Сегодня наводила порядки в шкафах и вот что на...,{'https://www.linkedin.com/in/ACoAADQoWmEBKHJ1...
113,Преподавание меня вдохновляет и прививает любо...,{'https://www.linkedin.com/in/ACoAABbILmkBvGgY...
114,Здравствуйте Меня зовут Даша. Я видеооператор/...,{'https://www.linkedin.com/in/ACoAACyx1w8BPlB4...


In [618]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 0 to 19
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   post_text  20 non-null     object
 1   users      20 non-null     object
dtypes: object(2)
memory usage: 480.0+ bytes
